In [ ]:
# ! pip install optuna
# ! pip install torch
# ! pip install pandas
# ! pip install numpy
# ! pip install matplotlib
# ! pip install kagglehub
# ! pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 2.4 MB/s  0:00:03m0:00:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 4.3 MB/s  0:00:08m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [scikit-learn] [scikit-learn]


In [10]:
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim as optim

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt

import optuna

/home/jatin/Documents/code/pytorch/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
# path = "/kaggle/input/fashionmnist"

import kagglehub

# Download latest version
path = kagglehub.dataset_download("zalando-research/fashionmnist")

print("Path to dataset files:", path)

In [ ]:
df = pd.read_csv(path + "/fashion-mnist_train.csv")

df.head()

In [ ]:
sample_df = df.sample(n=60000, random_state=42)

In [ ]:
X = sample_df.iloc[:, 1:].values
y = sample_df.iloc[:, :1].values


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
st_scaler = StandardScaler()

In [ ]:

X_train = st_scaler.fit_transform(X_train)
X_test = st_scaler.transform(X_test)

# Dataset Class

In [ ]:
# create custom dataset class

class FashionMNISTDataset(Dataset):

    def __init__(self, features, labels):
        
        self.features = torch.tensor(features, dtype=torch.float32).reshape(-1, 1, 28, 28) # converted 1D 784 rows into 28x28 2D rows, 
                                                                                        # 1 states that its a grayscale image, -1 is the batch size
        self.labels = torch.tensor(labels, dtype=torch.long)
        
        


    def __len__(self):
        return len(self.features)


    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [ ]:
train_dataset = FashionMNISTDataset(X_train, y_train)
test_dataset = FashionMNISTDataset(X_test, y_test)

In [ ]:
# loading data
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)


In [ ]:
class CNNModel(nn.Module):
    
    def __init__(self, input_features):
        
        super().__init__()
        
        # convolutions and pooling layer
        self.features = nn.Sequential(
            nn.Conv2d(
                in_channels = input_features,
                out_channels = 32,
                kernel_size = 3,
                padding = 'same' 
            ),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(
                in_channels = 32,
                out_channels = 64,
                kernel_size = 3,
                padding = 'same' 
            ),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*7*7 , 128),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            
            nn.Linear(64, 10)
        )
        
        
        
        
        
    def forward(self, x):
        
        x = self.features(x)
        x = self.classifier(x)
        
        return x
    

In [ ]:
learning_rate = 0.005
epochs = 100

model = CNNModel(1).to(device)

loss_function = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=learning_rate)




In [ ]:
total_epoch_loss = []

for epoch in range(epochs):
    print(f"----------- Epoch {epoch+1}/{epochs} -----------")

    for batch_features, batch_labels in train_loader:
        
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

        
        # forward pass
        outputs = model(batch_features)

        # loss calculation
        loss = loss_function(outputs, batch_labels.squeeze())
        total_epoch_loss.append(loss.item())

        # backprop

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

    print(f" avg epoch loss: {sum(total_epoch_loss)/len(total_epoch_loss)} ")

In [ ]:
        
# evaluation

model.eval()


# return accuracy

total = 0
correct = 0 
i = 1
cnt =0
with torch.no_grad():
    for batch_features, batch_labels in test_loader:

        cnt +=1
        
        # move data to gpu
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
            
        test_outputs = model(batch_features)

        _, predicted = torch.max(test_outputs, 1)
        
        
        
        total += batch_labels.shape[0]
        correct += (predicted == batch_labels.squeeze()).sum().item()


accuracy = 100 * correct / total

print(accuracy)